In [1]:
%run ThetaS.ipynb
# the firmware
data = open('ap1_v131.frm','rb').read()

In [275]:
# parse all sections
sections = Sections(data, 0x800, 0x200)

In [276]:
sections.reset()
keys_1 = sections.search_xor_keys(find_xor_key)

10136 sections/keys and 10013 distinct keys


In [277]:
def find_xor_key_2(_section):
    # try to find key by finding 0x00000000
    words = map(''.join, zip(*[iter(_section)]*4))
    # add best guess for key if value was 0xffffffff
    # look at unique keys
    unique_key = set(words)
    # for each key, if we found
    for word in list(words):
        if word in unique_key:
            continue
        if xor(word, '\xff\xff\xff\xff') in unique_key:
            key = xor(word, '\xff\xff\xff\xff')
            print 'found corresponding 0xf for key', binascii.hexlify(key)
            # reinforce key
            words.append(key)
    # now look at stats
    words = map(binascii.hexlify, words)
    freqs = Counter(words)
    key = freqs.most_common(1)[0][0]
    # unhex_key = binascii.unhexlify(key)
    return key, freqs


In [278]:
sections2 = Sections(data, 0x800, 0x200)
keys_2 = sections2.search_xor_keys(find_xor_key_2)

10136 sections/keys and 10013 distinct keys


In [279]:
# search for differences
if keys_1 != keys_2:
    print "different keys have been calculated"
else:
    print "no improvements in find_xor_key_2"

def show_keys_diff(ks1, ks2):
    for a, b in enumerate(zip(ks1, ks2)):
        if a != b:
            print 'key diff at %d: %s-%s' % (i, a, b)
            print a
            print hexdump(xor(sections.sections[i], binascii.unhexlify(a)))
            print b
            print hexdump(xor(sections.sections[i], binascii.unhexlify(a)))
    

no improvements in find_xor_key_2


In [280]:
interestings_sections = [0x2c00, 0x3000, 0x247ab0]

look_at = 0x247ab0
section = sections[look_at]
print section
start = hex(section.start)
print start

key = section.xor_key
print 'key:', section.hex_key
freqs = section.xor_freq
print hexdump(section.section_out)
for key, freq in freqs.items():
    out = xor(section.data[0xa0:0x100], key)
    if 'BEGIN RSA' in out:
        print binascii.hexlify(key)
        print hexdump(out)
        break

#print 'key:', sections2.section_keys[start]
#print hexdump(sections2.section_outs[start], start)

0x247a00
key: 723b75cd
0000  00 00 00 00 00 00 00 00 18 00 00 00 10 00 00 00   ................
0010  13 01 00 00 15 00 00 00 00 00 00 00 10 00 00 00   ................
0020  14 00 00 00 11 00 00 00 15 00 00 00 00 00 00 00   ................
0030  10 00 00 00 15 00 00 00 12 00 00 00 00 00 00 00   ................
0040  00 00 00 00 10 00 00 00 15 00 00 00 15 01 00 00   ................
0050  15 00 00 00 18 00 00 00 18 00 00 00 13 00 00 00   ................
0060  16 01 00 00 18 00 00 00 18 00 00 00 18 00 00 00   ................
0070  13 00 00 00 17 01 00 00 08 00 00 00 08 00 00 00   ................
0080  18 00 00 00 12 00 00 00 18 01 00 00 00 00 00 00   ................
0090  00 00 00 00 00 00 00 00 11 00 00 00 19 01 00 00   ................
00a0  30 00 00 00 30 00 00 00 00 00 00 00 11 00 00 00   0...0...........
00b0  3d 2d 2d 2d 3d 42 45 47 59 4e 20 52 43 41 20 50   =---=BEGYN RCA P
00c0  42 49 56 41 44 45 20 4b 55 59 2d 2d 3d 2d 2d 0a   BIVADE KUY--=--.
00d0  5d 49 49 45 7f 77 49 4

So the question is how to go from a partial XOR key (723b75cd) to a full XOR key recovery (723b75cd).

In [281]:
valid_key = '623b75cd'
print section.xor_freq.most_common(10)

[('723b75cd', 10), ('6a3b75cd', 7), ('673b75cd', 5), ('623b75cd', 4), ('633b75cd', 3), ('603b75cd', 2), ('7a3b75cd', 2), ('613b75cd', 2), ('423b75cd', 2), ('550b018c', 1)]


We can see similarities in the top 10. we might be able to retrieve the best 4-bytes KEY based on frequency analysis of the top 10 results ?

In [282]:

best, res = find_xor_key_3(section.data)
print "best meta-choice for key:", best
out = xor(section.data[0xa0:0x100], binascii.unhexlify(best))
print hexdump(out)

possible partial key 2 bits 6a3b75cd XOR3 18000000 FIXED 623b75cd
possible partial key 3 bits 673b75cd XOR3 15000000 FIXED 623b75cd
possible partial key 1 bits 623b75cd XOR3 10000000 FIXED 623b75cd
possible partial key 2 bits 633b75cd XOR3 11000000 FIXED 623b75cd
possible partial key 2 bits 603b75cd XOR3 12000000 FIXED 603b75cd
possible partial key 1 bits 7a3b75cd XOR3 08000000 FIXED 723b75cd
possible partial key 3 bits 613b75cd XOR3 13000000 FIXED 603b75cd
possible partial key 2 bits 423b75cd XOR3 30000000 FIXED 423b75cd
best meta-choice for key: 623b75cd
0000  20 00 00 00 20 00 00 00 10 00 00 00 01 00 00 00    ... ...........
0010  2d 2d 2d 2d 2d 42 45 47 49 4e 20 52 53 41 20 50   -----BEGIN RSA P
0020  52 49 56 41 54 45 20 4b 45 59 2d 2d 2d 2d 2d 0a   RIVATE KEY-----.
0030  4d 49 49 45 6f 77 49 42 41 41 4b 43 41 51 45 41   MIIEowIBAAKCAQEA
0040  7a 50 62 78 32 53 37 2b 78 6c 43 30 6a 78 35 45   zPbx2S7+xlC0jx5E
0050  76 35 4f 48 33 35 4b 79 58 74 38 78 37 30 74 41   v5OH35KyXt8x70tA

In [283]:
def get_best_bytes_pos(_data):
    # let's first identify section where the key is probably already found
    #from operator import itemgetter
    #keys = map(itemgetter(0), section.xor_freq.items())
    # frequency analysis per byte position
    bytes_1 = []
    bytes_2 = []
    bytes_3 = []
    bytes_4 = []
    for i in range(0, len(_data), 4):
        bytes_1.append(_data[i])
        bytes_2.append(_data[i+1])
        bytes_3.append(_data[i+2])
        bytes_4.append(_data[i+3])
    c_1 = Counter(bytes_1)
    c_2 = Counter(bytes_2)
    c_3 = Counter(bytes_3)
    c_4 = Counter(bytes_4)
    key = '%c%c%c%c' % ( c_1.most_common(1)[0][0], c_2.most_common(1)[0][0], c_3.most_common(1)[0][0], c_4.most_common(1)[0][0])
    print binascii.hexlify(key)
    res = [(binascii.hexlify(c), cnt) for c, cnt in c_1.items()]
    res.sort(key=lambda x: x[1], reverse=True)
    print res
    return key

res = get_best_bytes_pos(section.data)
if res != valid_key:
    print "FAIL"
    

723b75cd
[('72', 10), ('6a', 8), ('67', 6), ('62', 4), ('00', 3), ('1a', 3), ('27', 3), ('28', 3), ('30', 3), ('3a', 3), ('4f', 3), ('52', 3), ('61', 3), ('63', 3), ('0d', 2), ('11', 2), ('10', 2), ('13', 2), ('16', 2), ('20', 2), ('23', 2), ('29', 2), ('2b', 2), ('2c', 2), ('32', 2), ('42', 2), ('4d', 2), ('51', 2), ('50', 2), ('5b', 2), ('5a', 2), ('60', 2), ('7a', 2), ('01', 1), ('07', 1), ('08', 1), ('0b', 1), ('0a', 1), ('0c', 1), ('0f', 1), ('0e', 1), ('14', 1), ('18', 1), ('21', 1), ('25', 1), ('26', 1), ('2a', 1), ('2d', 1), ('2f', 1), ('2e', 1), ('31', 1), ('37', 1), ('36', 1), ('38', 1), ('3b', 1), ('49', 1), ('53', 1), ('55', 1), ('57', 1), ('56', 1), ('65', 1), ('64', 1), ('66', 1), ('68', 1), ('6b', 1)]
FAIL


In [284]:
section_a = sections[0x24D200]
section_b = sections[0x24D400]
print hexdump(section_a.section_out[-128:], 0x24D380)
print hexdump(section_b.section_out[:128], section_b.start)

# section_b.reset() 

# we expect 'E(60).checked' between the two sections
try_known_text_attack(sections, section_b.start, ').ch')


24d380  69 5d 29 2e 73 74 79 6c 65 2e 64 69 73 70 6c 61   i]).style.displa
24d390  79 3d 76 3f 27 62 6c 6f 63 6b 27 3a 27 6e 6f 6e   y=v?'block':'non
24d3a0  65 27 7d 7d 66 75 6e 63 74 69 6f 6e 20 63 28 29   e'}}function c()
24d3b0  7b 24 28 5b 31 2c 32 2c 33 2c 34 2c 31 35 2c 31   {$([1,2,3,4,15,1
24d3c0  36 2c 32 35 2c 32 36 2c 32 37 5d 2c 30 29 7d 77   6,25,26,27],0)}w
24d3d0  69 6e 64 6f 77 2e 6f 6e 6c 6f 61 64 3d 66 75 6e   indow.onload=fun
24d3e0  63 74 69 6f 6e 28 29 7b 45 28 33 30 29 2e 63 68   ction(){E(30).ch
24d3f0  65 63 6b 65 64 3d 31 3b 63 28 29 3b 45 28 36 30   ecked=1;c();E(60

24d400  5d 0e 17 11 11 43 1f 1c 10 1d 45 42 00 1d 53 2e   ]....C....EB..S.
24d410  1d 73 11 3a 1b 4e 1a 1c 17 54 54 2e 1d 52 11 15   .s.:.N...TT..R..
24d420  11 53 07 59 37 4f 1a 1f 1d 47 01 0b 15 54 1d 16   .S.Y7O...G...T..
24d430  1a 07 4f 1d 1b 43 01 14 11 4e 00 57 00 49 00 15   ..O..C...N.W.I..
24d440  11 1d 00 42 31 08 41 48 5d 0e 1d 17 1a 45 06 31   ...B1.AH]....E.1
24d450  20 6d 38 44 00 5

'd25faca4'

In [285]:
section_a = sections[0x24BE00]
section_b = sections[0x24C000]
print hexdump(section_a.section_out[-128:], 0x24BF80)
print hexdump(section_b.section_out[:128], section_b.start)

try_known_text_attack(sections, 0x24c000, 'A PRIVATE')


24bf80  64 72 65 73 73 3d 25 73 00 25 30 32 64 00 25 30   dress=%s.%02d.%0
24bf90  34 64 00 2d 2d 2d 2d 2d 42 45 47 49 4e 20 43 45   4d.-----BEGIN CE
24bfa0  52 54 49 46 49 43 41 54 45 2d 2d 2d 2d 2d 00 00   RTIFICATE-----..
24bfb0  00 12 1c ff 1f 58 1c ff 1f 72 1c ff 1f 92 1c ff   .....X...r......
24bfc0  1f b0 1c ff 1f cc 1c ff 1f e6 1c ff 1f 0c 1d ff   ................
24bfd0  1f 30 1d ff 1f 4e 1d ff 1f 2d 2d 2d 2d 2d 45 4e   .0...N...-----EN
24bfe0  44 20 43 45 52 54 49 46 49 43 41 54 45 2d 2d 2d   D CERTIFICATE---
24bff0  2d 2d 00 2d 2d 2d 2d 2d 42 45 47 49 4e 20 52 53   --.-----BEGIN RS

24c000  39 05 60 60 31 73 71 66 3d 05 7b 77 21 08 1d 1f   9.``1sqf=.{w!...
24c010  55 08 30 1f 55 08 1d 1f 3d 6b 74 12 2a 76 71 12   U.0.U...=kt.*vq.
24c020  28 77 79 64 39 71 75 12 33 60 69 1f 55 08 1d 1f   (wyd9qu.3`i.U...
24c030  78 08 1d 1f 55 08 72 77 3f 6c 7e 12 28 77 79 64   x...U.rw?l~.(wyd
24c040  39 71 75 12 33 60 69 1f 55 08 1d 1f 78 08 1d 1f   9qu.3`i.U...x...
24c050  55 08 75 7c 3c 0

'641ebce6'

In [286]:
zero = '\x00\x00\x00\x00'
tmp = zero
for i in range(0, len(section_a.data), 4):
    word = section_a.data[i:i+4]
    tmp = xor(tmp, word)

print binascii.hexlify(tmp), 'not a xor on previous bytes'

# first section at 0x800 is coded with '071969ed'
binascii.hexlify(xor(binascii.unhexlify('ffffffff'), binascii.unhexlify('071969ed')))
'f8e69612'

01dd84f9 not a xor on previous bytes


'f8e69612'

In [287]:
print data.count('UNITY'), data.index('UNITY'), hex(data.index('UNITY',1))
unity_offset1 = 0x0
unity_offset2 = data.index('UNITY',1)
print hexdump(data[0x2a5c00:0x2a5c00+0x40])
offset1 = 0x800
offset2 = 0x2a6400
print hex(unity_offset2+0x800)
print sections[offset1].hex_key, sections[offset2].hex_key
print sections[offset1 + 0x4900].hex_key, sections[offset2 + 0x4900].hex_key

2 0 0x2a5c00
0000  55 4e 49 54 59 20 46 49 4c 45 20 56 31 2e 31 30   UNITY FILE V1.10
0010  20 2f 20 52 49 43 4f 48 20 43 4f 4d 50 41 4e 59    / RICOH COMPANY
0020  00 24 dc 00 00 00 00 01 ff ff ff ff ff ff ff ff   .$..............
0030  ff ff ff ff ff ff ff ff ff ff ff ff ff ff ff ff   ................

0x2a6400
071969ed 071969ed
b9038ebe b9038ebe


It seems like the rolling xor keys is similar in both UNITY section.
 - In two different sub-unitySections, both random section 1 and N has the same XOR key.
 - The XOR key is generated by a non-data-feeded generator.

It is for our purpose, a static generator.

In [288]:
with open('unity.part1', 'wb') as fout:
    fout.write(data[:unity_offset2])

with open('unity.part2', 'wb') as fout:
    fout.write(data[unity_offset2:])

In [289]:
# save previous keys.
head = True
keys_1 = None
keys_2 = None
key_list = []
key_list.append(sections.sections[0].hex_key)
for i, _s in enumerate(sections.sections[1:]):
    if _s.hex_key == '071969ed':
        print 'rolling at i == %d, start = 0x%08x' % (i, _s.start)
        keys_1 = key_list
        key_list = []
    key_list.append(_s.hex_key)
print _s, hex(_s.start), _s.hex_key
keys_2 = key_list
print "keys_1 len:", len(keys_1)
print "keys_2 len:", len(keys_2)

rolling at i == 5421, start = 0x002a6400
<__main__.Section object at 0x7fcf7705c650> 0x4f3600 ffffffff
keys_1 len: 5422
keys_2 len: 4714


In [290]:
# save keys to file
with open('unity.xor.keys.part1', 'w') as fout:
    for k in keys_1:
        fout.write("%s\n" % k)

with open('unity.xor.keys.part2', 'w') as fout:
    for k in keys_2:
        fout.write("%s\n" % k)        


In [291]:
# we now need to produce the list of xor keys, and apply to both Sections
# load unity.xor.keys.part1 as our key list and make some adjustments.
keys = keys_1
unity1 = Sections(open('unity.part1', 'rb').read(), 0x800, 0x200)
unity2 = Sections(open('unity.part2', 'rb').read(), 0x800, 0x200) # offset is 0x2a6400
unity1.apply_xor(keys_1)
unity2.apply_xor(keys_2)

5418/5418 sections Xor-ed
4714/4714 sections Xor-ed


In [292]:
# to file for manual check
unity1.write_to_file('unity.part1.unxor')
unity2.write_to_file('unity.part2.unxor')

In [293]:
# add ignore here
ignore = [3, 4]
for i, (x1, x2) in enumerate(zip(keys, keys_2)):
    if x1 == x2:
        continue
    if i in ignore:
        continue
    print '@ ', hex(0x800 + i*0x200)
    comp(unity1.sections[i], unity2.sections[i])
    break

@  0xa00
section1 b72047a5 56e05715
0a00  00 2a 32 00 00 00 00 00 00 00 00 00 00 00 00 00   .*2.............
0a10  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
0a20  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
0a30  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
0a40  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
0a50  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
0a60  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
0a70  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
0a80  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
0a90  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
0aa0  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
0ab0  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
0ac0  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
0ad0  00 00 00 

In [294]:
# try other keys
# key = 'd0b25a10'
# print hexdump(unity1.sections[i].show( binascii.unhexlify( key )))
# print hexdump(unity2.sections[i].show( binascii.unhexlify( key )))

In [295]:
interestings_sections = [0x2c00, 0x3000, 0x247ab0, 0x24C400, 0x24C800]
misxor_sections = [0x24A400, 0x24B800, 0x24C000, 0x24C600]

#for section in sections.sections[2000:3000]:
for addr in misxor_sections[2:]:
    section = sections[addr]
    best1 = section.xor_freq.most_common(1)[0][0]
    ratio = 100*section.xor_freq.most_common(1)[0][1]/128
    print hex(section.start), "best at %d%%, nb others keys: %d" % (ratio, len(freqs))
    best3, freq3 = find_xor_key_3(section.data)
    #if best1 != best3:
    #    print 'Proposed is Different', best1, best3
    #    print hexdump(xor(section.data, binascii.unhexlify(best1)))
    #    print hexdump(xor(section.data, binascii.unhexlify(best3)))
    for k in freq3.elements():
        print hexdump(xor(section.data[:128], binascii.unhexlify(k)))


0x24c000 best at 6%, nb others keys: 100
no meta choices for this one
0000  39 20 27 22 31 56 36 24 3d 20 3c 35 21 2d 5a 5d   9 '"1V6$= <5!-Z]
0010  55 2d 77 5d 55 2d 5a 5d 3d 4e 33 50 2a 53 36 50   U-w]U-Z]=N3P*S6P
0020  28 52 3e 26 39 54 32 50 33 45 2e 5d 55 2d 5a 5d   (R>&9T2P3E.]U-Z]
0030  78 2d 5a 5d 55 2d 35 35 3f 49 39 50 28 52 3e 26   x-Z]U-55?I9P(R>&
0040  39 54 32 50 33 45 2e 5d 55 2d 5a 5d 78 2d 5a 5d   9T2P3E.]U-Z]x-Z]
0050  55 2d 32 3e 3c 20 27 22 31 56 36 24 3d 20 3c 35   U-2>< '"1V6$= <5
0060  21 2d 5a 5d 55 2d 77 5d 55 2d 5a 5d 3a 45 30 39   !-Z]U-w]U-Z]:E09
0070  36 20 32 3e 3b 52 2e 20 2c 45 33 50 28 52 3e 26   6 2>;R. ,E3P(R>&

0000  61 56 35 20 69 20 24 26 65 56 2e 37 79 5b 48 5f   aV5 i $&eV.7y[H_
0010  0d 5b 65 5f 0d 5b 48 5f 65 38 21 52 72 25 24 52   .[e_.[H_e8!Rr%$R
0020  70 24 2c 24 61 22 20 52 6b 33 3c 5f 0d 5b 48 5f   p$,$a" Rk3<_.[H_
0030  20 5b 48 5f 0d 5b 27 37 67 3f 2b 52 70 24 2c 24    [H_.['7g?+Rp$,$
0040  61 22 20 52 6b 33 3c 5f 0d 5b 48 5f 20 5b 48 5f